In [1]:
import os

In [2]:
%pwd

'c:\\Users\\frup00090410\\Mlops_project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\frup00090410\\Mlops_project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir: Path
    preprocessed_spilitted_data_path: Path
    model_path: Path
    batch_size: int
    epoch: int
    validation_split: float
    learning_rate: float
    beta_1: float
    beta_2: float

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, write_to_pickle

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_training_config(self) -> ModelTrainingConfig:
        config = self.config.data_preprocessing
        
        create_directories([config.root_dir])

        model_training_config = ModelTrainingConfig(
            root_dir=Path(config.root_dir),
            preprocessed_spilitted_data_path=Path(config.preprocessed_spilitted_data_path),
            model_path=Path(config.model_path),
            batch_size=self.params.BATCH_SIZE,
            epochs=self.params.EPOCHS,
            validation_split=self.params.VALIDATION_SPLIT,
            learning_rate=self.params.LEARNING_RATE,
            beta_1=self.params.BETA_1,
            beta_2=self.params.BETA_2
        )

        return model_training_config

In [ ]:
import os
import pandas as pd
import numpy as np
import string
import json
from cnnClassifier import logger
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from collections import Counter
import pickle